In [1]:
# Import all the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
import nltk
import os
import re
import string
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer
from get_tweets import get_tweets_by_search_term
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
pd.set_option("display.max_columns", None)

In [2]:
# Import NLTK lexicon
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('twitter_samples')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/brightkoech/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/brightkoech/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/brightkoech/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/brightkoech/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/brightkoech/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [3]:
# Fetch the Tweets
keywords = input("Enter keywords, hashtags separated by commas: ")
keywords = list(set(keywords.split(",")))
get_tweets_by_search_term(keywords)

,UserId,TweetId,tweet,location,created
0,Den,1526291378887708673,@stablekwon we want the Terra and Luna together! we don't want another project ... who else can ...,,16-May-2022
1,Luci,1526291378832953344,@BlizzardCS I thought I would let you all know that I did find my Authenticator. Thank you all f...,,16-May-2022
2,TOM$✨,1526291378724130830,like I gotta be more firm with my boundaries,,16-May-2022
3,Vihenda,1526291378686369798,@luhya_kidd 😂😂😂😂😂😂 you know it,,16-May-2022
4,Josh innit,1526291378598199298,When you hear the last 30 seconds of ‘fight the feeling’ by MacMiller for the first team https:/...,,16-May-2022
...,...,...,...,...,...
4995,Leonis Tord,1526291344540442624,@TeaTheKook Why must you hurt me this way?,,16-May-2022
4996,Tyler 💙🏳️‍🌈🧜‍♂️,1526291344536154112,"@1800callPaul @DilophosaurusXL This is royalty of the sea, and I will not stand for slander",Akimel O’odham/Hohokam,16-May-2022
4997,ً,1526291344523673600,he's so fucking cute it pains me - j https://t.co/Mdced2IHCz,,16-May-2022
4998,Devon,1526291344519471110,54 doubles 🤣🤣🤣 his OPS during this stretch has to be well over .1000 https://t.co/EafYMsGxHG,,16-May-2022


In [ ]:
# Save tweets to pickle file and reload to DF - Avoid API calls
this_file_path = os.path.abspath("__file__")
BASE_DIR = os.path.dirname(this_file_path)
tweet_pickle_file = os.path.join(BASE_DIR, "tweets", "tweet_list.pkl")
tweets_df = pd.read_pickle(tweet_pickle_file)
tweets_df.head()


In [ ]:
# Primary exploratory analysis
print('Dataset shape:', tweets_df.shape)
print('Dataset columns:', tweets_df.columns)
tweets_df.info()

In [ ]:
# Drop all columns except for Tweet and create a list of all words
tweets_only_df = tweets_df.drop(['UserId', 'TweetId', 'location', 'created'], axis=1)
tweets_only_df.head()

In [ ]:
# Remove http links from tweets
def remove_http_https(text):
    return re.sub(r'http\S+', '', text)

tweets_only_df['link_removed'] = tweets_only_df['tweet'].apply(lambda x: remove_http_https(x))
tweets_only_df.head()

In [ ]:
# Remove punctuations
#string.punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

tweets_only_df['tweet_punct'] = tweets_only_df['tweet'].apply(lambda x: remove_punct(x))
tweets_only_df.head()

In [ ]:
# Tokenize the tweets
def tokenize(text):
    text = re.split('\W+', text)
    return text

tweets_only_df['tweet_tokenized'] = tweets_only_df['tweet_punct'].apply(lambda x: tokenize(x))
tweets_only_df.head()

In [ ]:
# Remove stopwords
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopwords]
    return text

tweets_only_df['tweet_nonstop'] = tweets_only_df['tweet_tokenized'].apply(lambda x: remove_stopwords(x))
tweets_only_df.head()

In [ ]:
# Stemming and lemmatization
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

tweets_only_df['tweet_stemmed'] = tweets_only_df['tweet_nonstop'].apply(lambda x: stemming(x))

wn = nltk.WordNetLemmatizer()
def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
tweets_only_df['tweet_lemmatized'] = tweets_only_df['tweet_stemmed'].apply(lambda x: lemmatizer(x))
tweets_only_df.head()

In [ ]:
all_final_tweets = ' '.join(word for word in tweets_only_df['tweet_punct'])
fig, ax = plt.subplots(figsize=(30,30))
wordcloud_all_tweets = WordCloud(max_font_size=50, max_words=50, background_color="white").generate(all_final_tweets)
ax.imshow(wordcloud_all_tweets, interpolation='bilinear')
ax.set_title(f'Word Cloud of Top 100 words in 5000 Tweets Mentioning {keywords} ', fontsize=20)
ax.axis("off")